<a href="https://colab.research.google.com/github/jhonatanyara/SenalesySistemas/blob/main/PARCIAL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 **Parcial 2: Señales y Sistemas 2025-II**

---

**Estudiante:** Jhonatan Yara Lopez
**Profesor:** Andrés Marino Álvarez Meza, Ph.D.
**Departamento:** Ingeniería Eléctrica, Electrónica y Computación
**Universidad Nacional de Colombia - Sede Manizales**

---

### 📝 **Instrucciones Generales**
Este cuaderno contiene la solución computacional a los ejercicios planteados. Las simulaciones interactivas se ejecutan mediante **Streamlit**, implementando:
1.  **Demodulación AM (DSB-SC):** Filtrado ideal mediante FFT.
2.  **Sistemas Dinámicos:** Análisis de respuesta transitoria y analogías electromecánicas.


In [ ]:
# 1. Instalar librerías
!pip install streamlit numpy scipy matplotlib yt-dlp pydub

# 2. Descargar y configurar Cloudflared (Túnel estable)
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 3. Crear carpetas necesarias
import os
os.makedirs('pages', exist_ok=True)

In [ ]:
%%writefile 0_Inicio.py
import streamlit as st

# Configuración general
st.set_page_config(
    page_title="Parcial 2 - SyS",
    page_icon="🎓",
    layout="wide"
)

# Estilos CSS personalizados
st.markdown("""
    <style>
    .main-header {
        background-color: #f0f2f6;
        padding: 20px;
        border-radius: 10px;
        border-left: 8px solid #1E90FF;
        margin-bottom: 25px;
    }
    .main-title {
        color: #000000;
        font-family: 'Helvetica', sans-serif;
        font-weight: bold;
    }
    .info-text {
        color: #333333;
        font-size: 18px;
    }
    </style>

    <div class="main-header">
        <h1 class="main-title">Parcial #2: Señales y Sistemas</h1>
        <p class="info-text"><b>Estudiante:</b> Jhonatan Yara Lopez<br>
        <b>Profesor:</b> Andrés Marino Álvarez Meza, Ph.D.<br>
        <b>Universidad Nacional de Colombia</b></p>
    </div>
""", unsafe_allow_html=True)

st.markdown("""
### 📌 Instrucciones de Navegación

Este dashboard contiene la solución interactiva a los puntos del parcial. Utiliza el menú lateral (**Sidebar**) para navegar entre los ejercicios:

* **📻 1_Modulacion_AM**:
    * Carga de audio desde YouTube.
    * Modulación DSB-SC.
    * **Filtrado mediante FFT** y recuperación del mensaje.

* **⚙️ 2_Sistema_Masa_Resorte**:
    * Simulación de sistemas de 2do orden.
    * Comparativa **Lazo Abierto vs. Lazo Cerrado**.
    * Analogía con circuitos RLC.

---
👈 **Selecciona una página en el menú de la izquierda para comenzar.**
""")

st.sidebar.success("Selecciona un ejercicio arriba 👆")

## 📻 **Punto 1: Demodulador de Amplitud (DSB-SC)**

Se analiza un sistema de comunicación de Doble Banda Lateral con Portadora Suprimida. [cite_start]El proceso matemático implementado es el siguiente[cite: 173]:

### **1. Etapa de Modulación**
La señal transmitida $y(t)$ es el producto del mensaje $m(t)$ y la portadora:
$$y(t) = A_c m(t) \cos(2\pi f_c t + \theta_0)$$

### **2. Etapa de Demodulación (Mezclador)**
En el receptor, se multiplica la señal recibida nuevamente por un oscilador local. Asumiendo sincronización perfecta ($\theta_0 = 0$):
$$y_d(t) = y(t) \cdot \cos(2\pi f_c t)$$

[cite_start]Expandiendo trigonométricamente[cite: 195]:
$$y_d(t) = \frac{A_c}{2} m(t) [1 + \cos(4\pi f_c t)]$$

### **3. Etapa de Filtrado**
Para recuperar el mensaje $m(t)$, se aplica un **Filtro Pasa Bajas (LPF)** ideal. [cite_start]En este notebook, esto se implementa en el dominio de la frecuencia mediante la **Transformada Rápida de Fourier (FFT)**[cite: 167], eliminando la componente espectral centrada en $2f_c$.

$$m_{rec}(t) = \mathcal{F}^{-1} \{ Y_d(f) \cdot H_{LPF}(f) \}$$

In [ ]:
%%writefile pages/1_Modulacion_AM.py
import streamlit as st
import numpy as np
from scipy.fft import fft, ifft, fftshift, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import yt_dlp
import os
import subprocess

st.set_page_config(page_title="Punto 1: Modulación AM", page_icon="📻", layout="wide")

st.markdown("""
<div style="border-bottom: 2px solid #1E90FF; margin-bottom: 20px;">
    <h2 style="color: #1E90FF;">📻 Punto 1: Modulación y Demodulación (DSB-SC)</h2>
</div>
""", unsafe_allow_html=True)

# --- SIDEBAR ---
st.sidebar.header("🎛️ Parámetros de Señal")
url = st.sidebar.text_input("URL YouTube", "https://www.youtube.com/watch?v=5qap5aO4i9A")
start_s = st.sidebar.number_input("Segundo Inicio", value=30)
fc = st.sidebar.slider("Frecuencia Portadora (Hz)", 1000, 15000, 5000)
cutoff = st.sidebar.slider("Frecuencia Corte Filtro (Hz)", 1000, 8000, 4000)

# --- FUNCIONES ---
@st.cache_data
def get_audio(url, start, dur=5):
    """Descarga y procesa audio de YT"""
    out = "audio.wav"
    opts = {'format':'bestaudio/best', 'outtmpl':'temp', 'quiet':True}
    try:
        if os.path.exists(out): os.remove(out)
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download([url])
        # Convertir a WAV mono 44.1kHz con ffmpeg
        subprocess.run(
            ['ffmpeg', '-i', 'temp', '-ss', str(start), '-t', str(dur),
             '-ac', '1', '-ar', '44100', out, '-y'],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
        fs, data = wavfile.read(out)
        # Normalizar
        if data.dtype != np.float32:
            data = data.astype(np.float32) / np.max(np.abs(data))
        return fs, data
    except Exception as e:
        return None, None

def plot_tf(t, sig, fs, title, color):
    """Grafica Tiempo y Frecuencia"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))
    # Tiempo
    ax1.plot(t, sig, color=color, lw=0.8)
    ax1.set_title(f"Tiempo: {title}")
    ax1.grid(alpha=0.3)
    # Frecuencia
    N = len(sig)
    Y = fftshift(fft(sig))
    f = fftshift(fftfreq(N, 1/fs))
    ax2.plot(f, np.abs(Y)/N, color=color, lw=0.8)
    ax2.set_title(f"Espectro: {title}")
    ax2.set_xlim(-fs/2, fs/2)
    ax2.grid(alpha=0.3)
    st.pyplot(fig)

# --- APP ---
if st.sidebar.button("▶️ Iniciar Simulación"):
    with st.spinner("Descargando audio..."):
        fs, m_t = get_audio(url, start_s)

    if m_t is not None:
        N = len(m_t)
        t = np.linspace(0, 5, N)

        st.info("1. Señal Mensaje Original m(t)")
        st.audio(m_t, sample_rate=fs)
        plot_tf(t, m_t, fs, "Mensaje", "#1E90FF")

        # Modulación
        c_t = np.cos(2*np.pi*fc*t)
        y_t = m_t * c_t
        st.info("2. Señal Modulada (DSB-SC)")
        st.audio(y_t, sample_rate=fs)
        plot_tf(t, y_t, fs, "Modulada", "#28a745")

        # Demodulación (Mezcla)
        y_mix = y_t * c_t
        st.info("3. Señal Mezclada (Antes del Filtro)")
        plot_tf(t, y_mix, fs, "Mezclador", "#dc3545")

        # Filtro Ideal FFT
        Y_f = fft(y_mix)
        freqs = fftfreq(N, 1/fs)
        mask = np.abs(freqs) < cutoff
        m_rec = np.real(ifft(Y_f * mask)) * 2 # x2 ganancia

        st.success("4. Señal Recuperada (Filtrado Ideal)")
        st.audio(m_rec, sample_rate=fs)
        plot_tf(t, m_rec, fs, "Recuperada", "#1E90FF")
    else:
        st.error("Error al descargar audio.")
else:
    st.info("Presiona el botón en la barra lateral para comenzar.")

## ⚙️ **Punto 2: Sistema Masa-Resorte y Equivalente Eléctrico**

Se modela un sistema dinámico de segundo orden. [cite_start]A partir de la ecuación diferencial del sistema mecánico $F(t) = m\ddot{y} + c\dot{y} + ky$, se obtiene la función de transferencia en lazo abierto[cite: 235]:

$$G_{ol}(s) = \frac{1}{ms^2 + cs + k}$$

[cite_start]Llevándolo a la **forma canónica** de segundo orden[cite: 239]:
$$G(s) = \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}$$

### **Analogía Electromecánica (Serie)**
[cite_start]Para la simulación, se utilizan las siguientes equivalencias derivadas del circuito RLC serie [cite: 262-264]:

| Parámetro Mecánico | Parámetro Eléctrico | Relación Matemática |
| :--- | :--- | :--- |
| Masa ($m$) | Inductancia ($L$) | $L = \frac{1}{\omega_n^2 C}$ |
| Amortiguador ($c$) | Resistencia ($R$) | $R = \frac{2\zeta}{\omega_n C}$ |
| Resorte ($k$) | Capacitancia ($1/C$) | $\omega_n = \frac{1}{\sqrt{LC}}$ |

> **Nota:** Se asume un capacitor $C$ de valor fijo para calcular los demás componentes.

In [ ]:
%%writefile pages/2_Sistema_Masa_Resorte.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

st.set_page_config(page_title="Punto 2: Sistemas 2do Orden", page_icon="⚙️", layout="wide")

st.markdown("""
<div style="border-bottom: 2px solid #FFC107; margin-bottom: 20px;">
    <h2 style="color: #D39E00;">⚙️ Punto 2: Sistema Masa-Resorte y Equivalente RLC</h2>
</div>
""", unsafe_allow_html=True)

# --- SIDEBAR ---
st.sidebar.header("Parámetros del Sistema")
caso = st.sidebar.selectbox("Caso de Amortiguamiento", ["Subamortiguado (Oscila)", "Crítico", "Sobreamortiguado"])

if "Sub" in caso: zeta = st.sidebar.slider("Zeta (ζ)", 0.05, 0.95, 0.3)
elif "Crít" in caso: zeta = 1.0; st.sidebar.info("Zeta fijo en 1.0")
else: zeta = st.sidebar.slider("Zeta (ζ)", 1.05, 5.0, 1.5)

wn = st.sidebar.slider("Omega n (rad/s)", 1.0, 20.0, 5.0)
lazo = st.sidebar.radio("Configuración", ["Lazo Abierto", "Lazo Cerrado"])

# --- CÁLCULOS ---
# Mecánico (Normalizado m=1)
m = 1.0
k = (wn**2) * m
c = 2 * zeta * wn * m

# Eléctrico (Serie RLC, asumiendo C=1mF para calcular L y R)
C_el = 0.001
L_el = 1 / (wn**2 * C_el)
R_el = 2 * zeta * wn * L_el

# Funciones de Transferencia
# G(s) Open Loop = wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
num = [wn**2]
den = [1, 2*zeta*wn, wn**2]

if lazo == "Lazo Cerrado":
    # G_cl = G / (1+G) -> Denominador cambia: s^2 + 2*zeta*wn*s + 2*wn^2
    den_sys = [den[0], den[1], den[2] + num[0]]
    sys = signal.TransferFunction(num, den_sys)
    wn_eff = np.sqrt(den_sys[2])
    z_eff = den_sys[1]/(2*wn_eff)
else:
    sys = signal.TransferFunction(num, den)
    wn_eff = wn
    z_eff = zeta

# --- VISUALIZACIÓN ---
c1, c2, c3 = st.columns(3)
c1.metric("Resorte (k)", f"{k:.2f} N/m", f"Amortiguador: {c:.2f}")
c2.metric("Inductancia (L)", f"{L_el:.3f} H", f"Resistencia: {R_el:.2f} Ω")
c3.metric("Frecuencia Efectiva", f"{wn_eff:.2f} rad/s", f"Zeta Efectivo: {z_eff:.3f}")

st.markdown("---")

tab1, tab2, tab3 = st.tabs(["Respuesta al Escalón", "Diagrama de Bode", "Polos y Ceros"])

with tab1:
    t, y = signal.step(sys)
    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(t, y, lw=2, label=lazo)
    ax.set_title("Respuesta Transitoria")
    ax.set_xlabel("Tiempo (s)")
    ax.grid(True, alpha=0.5)
    ax.legend()
    st.pyplot(fig)

with tab2:
    w, mag, phase = signal.bode(sys)
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 4))
    ax1.semilogx(w, mag, color='purple')
    ax1.set_ylabel("Magnitud (dB)")
    ax1.grid(True, which="both", alpha=0.3)
    ax2.semilogx(w, phase, color='purple')
    ax2.set_ylabel("Fase (deg)")
    ax2.grid(True, which="both", alpha=0.3)
    st.pyplot(fig)

with tab3:
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(np.real(sys.poles), np.imag(sys.poles), marker='x', s=100, color='red', label='Polos')
    if len(sys.zeros) > 0:
        ax.scatter(np.real(sys.zeros), np.imag(sys.zeros), marker='o', s=100, facecolors='none', edgecolors='blue')
    ax.axhline(0, color='k', lw=1); ax.axvline(0, color='k', lw=1)
    ax.grid(True)
    ax.set_title("Mapa de Polos")
    st.pyplot(fig)

In [ ]:
# Ejecutar Streamlit en segundo plano
!streamlit run 0_Inicio.py &>/dev/null &

# Iniciar el túnel y mostrar la URL
import time
print("⏳ Iniciando Streamlit, por favor espera unos segundos...")
time.sleep(5) # Dar tiempo a que arranque streamlit

# Ejecutar el túnel Cloudflare
# Esto mostrará un enlace que termina en .trycloudflare.com
!cloudflared tunnel --url http://localhost:8501